In [100]:
import importlib
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scripts import forecast_pipeline as fp
from scripts import data_fetching as df
importlib.reload(fp)

<module 'scripts.forecast_pipeline' from '/Users/vasilisvc6/Documents/Le grand cormoran project/scripts/forecast_pipeline.py'>

## Current Date

In [77]:
date = '2024-12-22'
ped_start = '2024-06-01'
ped_end = '2024-08-30'

## Macroeconomic Indicators

In [67]:
macroeconomic = fp.macro_forecast(date)

In [68]:
macroeconomic

,CPI,Unemployment Rate,CPI_lag_10,Unemployment Rate_lag_7,Unemployment Rate_lag_10,Unemployment Rate_lag_14,Unemployment Rate_lag_30,Bond Yields_lag_7,Bond Yields_lag_10,Bond Yields_lag_14,Bond Yields_lag_30
2024-12-01,3.746553,6.700000,3.789761,6.746667,6.766667,6.793333,6.900000,3.188800,3.2005,3.2161,3.2785
2024-12-02,3.742641,6.696774,3.785440,6.740000,6.760000,6.786667,6.893333,3.184900,3.1966,3.2122,3.2746
2024-12-03,3.738729,6.693548,3.781119,6.733333,6.753333,6.780000,6.886667,3.181000,3.1927,3.2083,3.2707
2024-12-04,3.734817,6.690323,3.776799,6.726667,6.746667,6.773333,6.880000,3.177100,3.1888,3.2044,3.2668
2024-12-05,3.730904,6.687097,3.772478,6.720000,6.740000,6.766667,6.873333,3.173200,3.1849,3.2005,3.2629
2024-12-06,3.726992,6.683871,3.768157,6.713333,6.733333,6.760000,6.866667,3.169300,3.1810,3.1966,3.2590
2024-12-07,3.723080,6.680645,3.763836,6.706667,6.726667,6.753333,6.860000,3.165400,3.1771,3.1927,3.2551
2024-12-08,3.719168,6.677419,3.759515,6.700000,6.720000,6.746667,6.853333,3.161500,3.1732,3.1888,3.2512
2024-12-09,3.715256,6.674194,3.755195,6.696774,6.713333,6.740000,6.846667,3.165601,3.1693,3.1849,3.2473
2024-12-10,3.711344,6.670968,3.750874,6.693548,6.706667,6.733333,6.840000,3.169702,3.1654,3.1810,3.2434


## Weather Forecast

In [ ]:
montreal = fp.weather_forecast(date)

In [11]:
montreal

,tavg,wspd
time,,
2024-12-01,-0.9,15.7
2024-12-02,-2.0,12.1
2024-12-03,-2.4,8.4
2024-12-04,-2.0,15.6
2024-12-05,-0.2,17.1
2024-12-06,-7.3,21.4
2024-12-07,-8.0,8.4
2024-12-08,0.1,19.0
2024-12-09,-6.0,14.7


## Holidays forecast

In [78]:
holiday_feature = fp.holiday_feature(date)
holiday_feature

,holiday_type_2,before_holiday
2024-12-23,0,1
2024-12-24,1,1
2024-12-25,1,0
2024-12-26,0,0
2024-12-27,0,0
2024-12-28,0,1
2024-12-29,0,1
2024-12-30,0,1
2024-12-31,1,1
2025-01-01,1,0


## Pedestranization

In [15]:
pedestrian_feature = fp.pedestrianization(date, ped_start='2025-06-01', ped_end='2025-08-30')
pedestrian_feature

,is_pedestrian
2024-12-01,0
2024-12-02,0
2024-12-03,0
2024-12-04,0
2024-12-05,0
2024-12-06,0
2024-12-07,0
2024-12-08,0
2024-12-09,0
2024-12-10,0


## Time based features

In [16]:
time_features = fp.time_features(date)
time_features

,is_weekend,day_of_week_4,quarter_1,quarter_3
2024-12-01,1,0,0,0
2024-12-02,0,0,0,0
2024-12-03,0,0,0,0
2024-12-04,0,0,0,0
2024-12-05,0,0,0,0
2024-12-06,0,1,0,0
2024-12-07,1,0,0,0
2024-12-08,1,0,0,0
2024-12-09,0,0,0,0
2024-12-10,0,0,0,0


## Forecast data pipeline

In [79]:
data = fp.forecast_pipe(date, ped_start, ped_end)

In [80]:
data

,closed,holiday_type_2,is_pedestrian,is_weekend,CPI,Unemployment Rate,tavg,wspd,quarter_1,quarter_3,...,Unemployment Rate_lag_7,Unemployment Rate_lag_10,Unemployment Rate_lag_14,Unemployment Rate_lag_30,Bond Yields_lag_7,Bond Yields_lag_10,Bond Yields_lag_14,Bond Yields_lag_30,before_holiday,tavg_weekend
2024-12-23,0,0,0,0,3.660486,6.629032,-11.1,7.6,0,0,...,6.651613,6.661290,6.674194,6.753333,3.223018,3.210714,3.194309,3.192700,1,-0.0
2024-12-24,0,1,0,0,3.656574,6.625806,-8.9,7.6,0,0,...,6.648387,6.658065,6.670968,6.746667,3.227119,3.214815,3.198411,3.188800,1,-0.0
2024-12-25,0,1,0,0,3.652661,6.622581,-14.7,4.6,0,0,...,6.645161,6.654839,6.667742,6.740000,3.231220,3.218916,3.202512,3.184900,0,-0.0
2024-12-26,0,0,0,0,3.648749,6.619355,-13.3,3.2,0,0,...,6.641935,6.651613,6.664516,6.733333,3.235321,3.223018,3.206613,3.181000,0,-0.0
2024-12-27,0,0,0,0,3.644837,6.616129,-10.6,6.2,0,0,...,6.638710,6.648387,6.661290,6.726667,3.239422,3.227119,3.210714,3.177100,0,-0.0
2024-12-28,0,0,0,1,3.640925,6.612903,-6.2,10.5,0,0,...,6.635484,6.645161,6.658065,6.720000,3.243523,3.231220,3.214815,3.173200,1,-6.2
2024-12-29,0,0,0,1,3.637013,6.609677,1.3,10.6,0,0,...,6.632258,6.641935,6.654839,6.713333,3.247625,3.235321,3.218916,3.169300,1,1.3
2024-12-30,0,0,0,0,3.633101,6.606452,6.5,21.7,0,0,...,6.629032,6.638710,6.651613,6.706667,3.251726,3.239422,3.223018,3.165400,1,0.0
2024-12-31,0,1,0,0,3.629188,6.603226,2.6,16.0,0,0,...,6.625806,6.635484,6.648387,6.700000,3.255827,3.243523,3.227119,3.161500,1,0.0
2025-01-01,0,1,0,0,3.625276,6.600000,0.6,14.5,1,0,...,6.622581,6.632258,6.645161,6.696774,3.259928,3.247625,3.231220,3.165601,0,0.0


In [101]:
coffee_predictions = fp.load_sales_model_and_forecast('sales_models/xgb_model_Coffee.pkl', data, date)

In [102]:
without_coffee_predictions = fp.load_sales_model_and_forecast('sales_models/xgb_model_Without_Coffee.pkl', data, date)

In [103]:
food_predictions = fp.load_sales_model_and_forecast('sales_models/xgb_model_Food.pkl', data, date)

In [104]:
coffee_predictions

,sales
2024-12-23,576.276428
2024-12-24,579.590759
2024-12-25,622.882568
2024-12-26,567.718811
2024-12-27,631.045410
2024-12-28,706.237183
2024-12-29,805.075012
2024-12-30,566.072937
2024-12-31,588.136719
2025-01-01,594.064880


In [105]:
food_predictions

,sales
2024-12-23,372.665436
2024-12-24,396.027832
2024-12-25,386.490570
2024-12-26,346.240295
2024-12-27,379.190125
2024-12-28,481.864288
2024-12-29,484.571747
2024-12-30,339.887665
2024-12-31,359.676666
2025-01-01,364.433899


In [106]:
without_coffee_predictions

,sales
2024-12-23,141.198563
2024-12-24,153.376205
2024-12-25,124.447906
2024-12-26,141.198563
2024-12-27,155.806519
2024-12-28,215.245087
2024-12-29,230.385162
2024-12-30,154.270599
2024-12-31,154.807190
2025-01-01,138.446320
